# Custom coordinates static time-series

In this notebook, we create *static time-series* regarding the coordinates (longitude and latitude) for the administrative regions of the following countries:

- Yemen
- Nigeria
- Burkina Faso
- Syria

With the term *static*, we mean time-series that not change over time. We select a certain starting year as reference to create our time-series until another reference year.

In [1]:
import geopandas as gpd
import pandas as pd
import ntpath
import glob

In [2]:
# Add the python path to the folder containing some custom packages.
import sys
sys.path.insert(0, "../../packages/")
from TsIP.TsIP import TsIP

In [3]:
# Define the path where the results arising from this analysis will be saved.
PATH_TO_SAVE_DATA = "./output_timeseries/"

In [4]:
# Read the shapefiles of each country.
path = "../Shape Files/shapefiles/"
all_folders = glob.glob(path + "*")

dfs = []

for folder in all_folders:
    country = ntpath.basename(folder).split(".")[0]
    all_files = glob.glob(path + country + "/administrative level 1/*.shp")
    for filename in all_files:
        df = gpd.read_file(filename)
        df["Country"] = country
        dfs.append(df)

In [5]:
# Concatenate data.
df = pd.concat(dfs, axis = 0, ignore_index = True)
df.head()

,admin,geometry,Country
0,Boucle-Du-Mouhoun,"POLYGON ((-3.18158 13.67744, -3.18109 13.67721...",Burkina Faso
1,Cascades,"POLYGON ((-5.38923 11.00086, -5.37382 10.97698...",Burkina Faso
2,Centre,"POLYGON ((-1.60508 12.67250, -1.58739 12.67057...",Burkina Faso
3,Centre-Est,"POLYGON ((-0.30646 12.55612, -0.30573 12.55588...",Burkina Faso
4,Centre-Nord,"MULTIPOLYGON (((-0.76975 12.55062, -0.77002 12...",Burkina Faso


In [6]:
def coordinates(row):
    lon = row.geometry.centroid.x
    lat = row.geometry.centroid.y
    return pd.Series([lon, lat])

In [7]:
# Change the name of adminstrata feature.
df.rename(columns = {"admin": "AdminStrata"}, inplace = True)
# Create coordinates features.
df[["Lon", "Lat"]] = df.apply(coordinates, axis = 1)
# Delete geometry feature.
df.drop(columns = "geometry", inplace = True)
df.head()

,AdminStrata,Country,Lon,Lat
0,Boucle-Du-Mouhoun,Burkina Faso,-3.488905,12.535927
1,Cascades,Burkina Faso,-4.570349,10.354484
2,Centre,Burkina Faso,-1.499961,12.325874
3,Centre-Est,Burkina Faso,-0.186052,11.607598
4,Centre-Nord,Burkina Faso,-0.973623,13.271074


In [8]:
# Create multi-index columns dataframe.
df = df.set_index(["Country", "AdminStrata"]).unstack().unstack().reorder_levels([2, 1, 0]).to_frame().sort_index(0).transpose().dropna(axis = 1)
df.columns.set_names("Indicator", level = 2, inplace = True)
df

Country          Burkina Faso                                            \
AdminStrata Boucle-Du-Mouhoun             Cascades               Centre   
Indicator                 Lat       Lon        Lat       Lon        Lat   
0                   12.535927 -3.488905  10.354484 -4.570349  12.325874   

Country                                                           ...  \
AdminStrata           Centre-Est           Centre-Nord            ...   
Indicator         Lon        Lat       Lon         Lat       Lon  ...   
0           -1.499961  11.607598 -0.186052   13.271074 -0.973623  ...   

Country          Yemen                                                         \
AdminStrata     Sa'ada                Sana'a               Shabwah              
Indicator          Lat        Lon        Lat        Lon        Lat        Lon   
0            17.089972  43.920084  15.334488  44.347569  14.762417  46.923133   

Country                                                  
AdminStrata    Socotra                 Taizz             
Indicator          Lat        Lon        Lat        Lon  
0            12.488974  53.805239  13.416517  43.778161  

[1 rows x 172 columns]

In [9]:
# Define reference dates to build the time-series.
FIRST_YEAR = "2018"
LAST_YEAR = "2021"

In [10]:
# Create static time-series of the coordinates.
new_index = pd.date_range(FIRST_YEAR, LAST_YEAR, freq = "D")
df = df.loc[df.index.repeat(len(new_index))]
df.index = new_index
df.index.name = "Datetime"
df.head()

Country          Burkina Faso                                            \
AdminStrata Boucle-Du-Mouhoun             Cascades               Centre   
Indicator                 Lat       Lon        Lat       Lon        Lat   
Datetime                                                                  
2018-01-01          12.535927 -3.488905  10.354484 -4.570349  12.325874   
2018-01-02          12.535927 -3.488905  10.354484 -4.570349  12.325874   
2018-01-03          12.535927 -3.488905  10.354484 -4.570349  12.325874   
2018-01-04          12.535927 -3.488905  10.354484 -4.570349  12.325874   
2018-01-05          12.535927 -3.488905  10.354484 -4.570349  12.325874   

Country                                                           ...  \
AdminStrata           Centre-Est           Centre-Nord            ...   
Indicator         Lon        Lat       Lon         Lat       Lon  ...   
Datetime                                                          ...   
2018-01-01  -1.499961  11.607598 -0.186052   13.271074 -0.973623  ...   
2018-01-02  -1.499961  11.607598 -0.186052   13.271074 -0.973623  ...   
2018-01-03  -1.499961  11.607598 -0.186052   13.271074 -0.973623  ...   
2018-01-04  -1.499961  11.607598 -0.186052   13.271074 -0.973623  ...   
2018-01-05  -1.499961  11.607598 -0.186052   13.271074 -0.973623  ...   

Country          Yemen                                                         \
AdminStrata     Sa'ada                Sana'a               Shabwah              
Indicator          Lat        Lon        Lat        Lon        Lat        Lon   
Datetime                                                                        
2018-01-01   17.089972  43.920084  15.334488  44.347569  14.762417  46.923133   
2018-01-02   17.089972  43.920084  15.334488  44.347569  14.762417  46.923133   
2018-01-03   17.089972  43.920084  15.334488  44.347569  14.762417  46.923133   
2018-01-04   17.089972  43.920084  15.334488  44.347569  14.762417  46.923133   
2018-01-05   17.089972  43.920084  15.334488  44.347569  14.762417  46.923133   

Country                                                  
AdminStrata    Socotra                 Taizz             
Indicator          Lat        Lon        Lat        Lon  
Datetime                                                 
2018-01-01   12.488974  53.805239  13.416517  43.778161  
2018-01-02   12.488974  53.805239  13.416517  43.778161  
2018-01-03   12.488974  53.805239  13.416517  43.778161  
2018-01-04   12.488974  53.805239  13.416517  43.778161  
2018-01-05   12.488974  53.805239  13.416517  43.778161  

[5 rows x 172 columns]

In [11]:
# Now save the time-series of each country keeping as indeces for each time-series the own first and last index.
def save(group, name):
    country = group.name
    group = group[country]
    # Adjust time-series group.
    first_idx = group.first_valid_index()
    last_idx = group.last_valid_index()
    group = group.loc[first_idx:last_idx]
    # Save.
    group.to_csv(PATH_TO_SAVE_DATA + country + "/" + name + ".csv", index_label = False)

In [12]:
# Save the static time-series.
df.groupby(level = 0, axis = 1).apply(lambda x: save(x, name = "wfp_coordinates"));

In [13]:
# Plot static time-series.
TsIP(df).interactive_plot_df(matplotlib = False, style = "lines", yaxis = "Coordinates")

RadioButtons(description='Select:', options=('Time-series', 'Missing values'), value='Time-series')

Output()